In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Sickle_Cell_Anemia/GSE41575'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Role of platelet micrornas in sickle cell disease"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: Meg-01 cells'], 1: ['genotype/variation: miR1225 overexpression', 'genotype/variation: control']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
if "gene expression" in "!Series_title".lower() or "microarray" in "!Series_title".lower():
    is_gene_available = True

# Identify keys for Sickle_Cell_Anemia, age, and gender in the Sample Characteristics Dictionary
sample_characteristics = {
    0: ['cell line: Meg-01 cells'],
    1: ['genotype/variation: miR1225 overexpression', 'genotype/variation: control']
}

# Sickle_Cell_Anemia trait analysis
trait_values = sample_characteristics.get(1)
if trait_values and len(set(trait_values)) > 1:
    trait_row = 1

# Implement conversion functions
def convert_trait(value):
    if ':' in value:
        value = value.split(':')[1].strip()
    if value.lower() in ['mir1225 overexpression', 'overexpression']:
        return 1
    elif value.lower() == 'control':
        return 0
    return None

# As age and gender are not listed in sample_characteristics, data is unavailable
age_row = None
gender_row = None

# Conversion functions for age and gender (not applicable but implemented to fulfill requirements)
def convert_age(value):
    return None

def convert_gender(value):
    return None

# Save cohort information
save_cohort_info('GSE41575', './preprocessed/Sickle_Cell_Anemia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction (if applicable)
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Sickle_Cell_Anemia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Sickle_Cell_Anemia/trait_data/GSE41575.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1019586': [1], 'GSM1019587': [1], 'GSM1019588': [1], 'GSM1019589': [1], 'GSM1019590': [1], 'GSM1019591': [0], 'GSM1019592': [0], 'GSM1019593': [0], 'GSM1019594': [0], 'GSM1019595': [0]}
